<a href="https://colab.research.google.com/github/HeNeos/Mechanical/blob/master/Finite_Element_Analysis/Python_code_for_%22Matlab-Guide-to-Finite-Elements%22/Beam_Element.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

In [2]:
def NaiveMultiply(A,B):
    C = np.zeros((A.shape[0],B.shape[1]))
    for i in range(0,C.shape[0]):
        for j in range(0,C.shape[1]):
            aux = 0
            for k in range(0,A.shape[1]):
                aux += A[i][k]*B[k][j]
            C[i][j] = aux
    return C
 
def nextPowerofTwo(n):
    return int(2**(ceil(log2(n))))
 
def ModifyMatrix(A):
    newA = A
    if(A.shape[1]%2 == 1):
        aux = np.zeros((A.shape[0],A.shape[1]+1))
        aux = np.insert(A,A.shape[1],0,axis=1)
        newA = aux
    if(newA.shape[0]%2 == 1):
        aux = np.zeros((newA.shape[0]+1,newA.shape[1]))
        aux = np.insert(newA,newA.shape[0],0,axis=0)
        newA = aux
    return newA
 
def FastMultiply(oldA,oldB):
    rows = oldA.shape[0]
    columns = oldB.shape[1]
    if(rows <= 2 or columns <=2 or oldA.shape[1] <= 2 or oldB.shape[0] <= 2):
        return np.matmul(oldA,oldB)
    
    A = ModifyMatrix(oldA)
    B = ModifyMatrix(oldB)
    N1 = A.shape[0]
    N2 = A.shape[1]
    N3 = B.shape[0]
    N4 = B.shape[1]
    
 
    a = A[0:N1//2,0:N2//2]
    b = A[0:N1//2,N2//2:N2//2+N2//2]
    c = A[N1//2:N1//2+N1//2,0:N2//2]
    d = A[N1//2:N1//2+N1//2,N2//2:N2//2+N2//2]
    
    e = B[0:N3//2,0:N4//2]
    f = B[0:N3//2,N4//2:N4//2+N4//2]
    g = B[N3//2:N3//2+N3//2,0:N4//2]
    h = B[N3//2:N3//2+N3//2,N4//2:N4//2+N4//2]
    
    
    p1 = FastMultiply(a,(f-h))
    p3 = FastMultiply((c+d),e)
    p2 = FastMultiply((a+b),h)
    p4 = FastMultiply(d,(g-e))
    p5 = FastMultiply((a+d),(e+h))
    p6 = FastMultiply((b-d),(g+h))
    p7 = FastMultiply((a-c),(e+f))
    
    
    C = np.zeros((rows,columns))
    
    c11 = p5 + p4 - p2 + p6
    c12 = p1 + p2
    c21 = p3 + p4
    c22 = p1 + p5 - p3 - p7
    
 
    
    for i in range(0,N1//2):
        for j in range(0,N4//2):
            C[i][j] = c11[i][j]
            if(j + N4//2 < columns):
                C[i][j+N4//2] = c12[i][j]
            if(i + N1//2 < rows):
                C[i+N1//2][j] = c21[i][j]
                if(j + N4//2 < columns):
                    C[i+N1//2][j+N4//2] = c22[i][j]
    return C

In [3]:
def conjugate_grad(A, b, x=None):
    n = b.shape[0]
    if not x:
        x = np.ones((n,1))
    r = np.dot(A, x) - b
    p = - r
    r_k_norm = FastMultiply(np.transpose(r), r)
    for i in range(2*n):
        Ap = np.dot(A, p)
        
        alpha = r_k_norm /FastMultiply(np.transpose(p), Ap)
        x += alpha * p
        r += alpha * Ap
        r_kplus1_norm = FastMultiply(np.transpose(r), r)
        beta = r_kplus1_norm / r_k_norm
        r_k_norm = r_kplus1_norm
        p = beta * p - r
    return x

In [23]:
NodesCondition = []
ForcesCondition = []

def UBoundaryCondition(nU,u,i):
    nU[i][0] = u
    NodesCondition.append(i)
 
def FBoundaryCondition(nF,f,i):
    nF[i][0] += f
    ForcesCondition.append(i)

def DFBoundaryCondition(nF,w,e):
    f = Elements[e][0]
    s = Elements[e][1]
    l = L[e]
    FBoundaryCondition(nF,w*l/2,2*f+0)
    FBoundaryCondition(nF,w*l*l/12,2*f+1)
    FBoundaryCondition(nF,w*l/2,2*s+0)
    FBoundaryCondition(nF,-w*l*l/12,2*s+1)

def ElementStiffness(E, I, L):
    aux = np.zeros((4,4))
    s1 = np.array([12,6*L,-12,6*L])
    s2 = np.array([6*L,4*L*L,-6*L,2*L*L])
    for i in range(0,2):
        for j in range(0,4):
            aux[2*i][j] = s1[j]
            aux[2*i+1][j] = s2[j]
            if(i == 1):
                aux[2*i][j] *= -1

    aux[3][1] = 2*L*L
    aux[3][3] = 4*L*L

    aux = aux*E*I/(L**3)
    return aux

def AssemblyStiffness(nStiffnessMatrix,k,i,j):
    for p in range(0,2):
        for m in range(0,2):
            nStiffnessMatrix[2*i+p][2*i+m] += k[p][m]
            nStiffnessMatrix[2*i+p][2*j+m] += k[p][2+m]
            nStiffnessMatrix[2*j+p][2*i+m] += k[p+2][m]
            nStiffnessMatrix[2*j+p][2*j+m] += k[p+2][2+m]
 
def Initialize(nStiffnessMatrix,nU,nF):
    for i in range(0,Nodes):
        nU[i][0] = 0
        nF[i][0] = 0
    for i in range(0,NumberOfElement):
        AssemblyStiffness(nStiffnessMatrix,K[i],int(Elements[i][0]),int(Elements[i][1]))

def TMatrix(nT,i,angle):
    nT[2*i][2*i] = np.cos(angle)
    nT[2*i][2*i+1] = np.sin(angle)
    nT[2*i+1][2*i] = -np.sin(angle)
    nT[2*i+1][2*i+1] = np.cos(angle)

def ApplyT(nStiffnessMatrix, nT):
    return FastMultiply(FastMultiply(nT,nStiffnessMatrix),np.transpose(nT))

def PreSolvingStiffness(nStiffnessMatrix):
    nsize = Nodes-len(NodesCondition)
    newStiffness = np.zeros((nsize,nsize))
    contr = -1
    for i in range(0,Nodes):
        contc = -1
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        for j in range(0,Nodes):
            flagc = False
            for k in range(0,len(NodesCondition)):
                if(j == NodesCondition[k]):
                    flagc = True
                    break
            if(flagc):
                continue
            contc += 1
            newStiffness[contr][contc] = nStiffnessMatrix[i][j]
    return newStiffness
 
 
def PreSolvingF(nF,nS,nU):
    nsize = Nodes-len(NodesCondition)
    newF = np.zeros(nsize).reshape(nsize,1)
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            for k in range(0,Nodes):
                nF[k][0] = nF[k][0]-nS[k][i]*nU[i][0]
            continue
 
            
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        newF[contr][0] = nF[i][0]
    
    return newF
                      
 
def Solve(nStiffnessMatrix,nU,nF):
    newStiffness = PreSolvingStiffness(nStiffnessMatrix)
    newF = PreSolvingF(nF,nStiffnessMatrix,nU)
    u = conjugate_grad(newStiffness,newF)    
    contr = -1
    for i in range(0,Nodes):
        flagr = False
        for k in range(0,len(NodesCondition)):
            if(i == NodesCondition[k]):
                flagr = True
                break
        if(flagr):
            continue
        contr += 1
        nU[i][0] = u[contr][0]
    nnF = FastMultiply(StiffnessMatrix,nU)
    return nU,nnF

In [24]:
NodesCondition = []
Nodes = 4
Nodes *= 2
NumberOfElement = 3

E = 2.10e5 #MPA
K = []
w = -7 #N/mm
I = 5e-6*1e12 #mm^4
PosNodes = 1e3*np.array([0,3,7,9])
Elements = np.array([(0,1),(1,2),(2,3)])
L = []
for i in range(NumberOfElement):
    L.append(PosNodes[Elements[i][1]] - PosNodes[Elements[i][0]])
L = np.array(L)

for i in range(0,NumberOfElement):
    aux = ElementStiffness(E,I,L[i])
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(y=0,phi=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en Y
UBoundaryCondition(U,0,2*1+0) #Nodo 1 en Y
UBoundaryCondition(U,0,2*2+0) #Nodo 2 en Y
UBoundaryCondition(U,0,2*3+0) #Nodo 3 en Y
UBoundaryCondition(U,0,2*3+1) #Nodo 3 en phi


DFBoundaryCondition(F,w,1) #w distribuido en el elemento 1
#FBoundaryCondition(F,-14e3,2*1+0) #Nodo 1 en Y
#FBoundaryCondition(F,-14e3,2*2+0) #Nodo 2 en Y
#FBoundaryCondition(F,-9.33e6,2*1+1) #Nodo 1 en phi
#FBoundaryCondition(F,9.33e6,2*2+1) #Nodo 2 en phi

U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 4.66666667e+02  7.00000000e+05 -4.66666667e+02  7.00000000e+05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 7.00000000e+05  1.40000000e+09 -7.00000000e+05  7.00000000e+08
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-4.66666667e+02 -7.00000000e+05  6.63541667e+02 -3.06250000e+05
  -1.96875000e+02  3.93750000e+05  0.00000000e+00  0.00000000e+00]
 [ 7.00000000e+05  7.00000000e+08 -3.06250000e+05  2.45000000e+09
  -3.93750000e+05  5.25000000e+08  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.96875000e+02 -3.93750000e+05
   1.77187500e+03  1.18125000e+06 -1.57500000e+03  1.57500000e+06]
 [ 0.00000000e+00  0.00000000e+00  3.93750000e+05  5.25000000e+08
   1.18125000e+06  3.15000000e+09 -1.57500000e+06  1.05000000e+09]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -1.57500000e+03 -1.57500000e+06  1.57500000e+03 -1.57500000e+06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 

# Problema 7.1

In [28]:
NodesCondition = []
Nodes = 3
Nodes *= 2
NumberOfElement = 2

E = 2e5 #MPA
K = []
I = 7e-4*1e12 #mm^4
PosNodes = 1e3*np.array([0,3.5,5.5])
Elements = np.array([(0,1),(1,2)])
L = []
for i in range(NumberOfElement):
    L.append(PosNodes[Elements[i][1]] - PosNodes[Elements[i][0]])
L = np.array(L)

for i in range(0,NumberOfElement):
    aux = ElementStiffness(E,I,L[i])
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(y=0,phi=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en Y
UBoundaryCondition(U,0,2*1+0) #Nodo 1 en Y
UBoundaryCondition(U,0,2*2+0) #Nodo 2 en Y


FBoundaryCondition(F,-15e6,2*1+1) #M en Nodo 1

U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 3.91836735e+04  6.85714286e+07 -3.91836735e+04  6.85714286e+07
   0.00000000e+00  0.00000000e+00]
 [ 6.85714286e+07  1.60000000e+11 -6.85714286e+07  8.00000000e+10
   0.00000000e+00  0.00000000e+00]
 [-3.91836735e+04 -6.85714286e+07  2.49183673e+05  1.41428571e+08
  -2.10000000e+05  2.10000000e+08]
 [ 6.85714286e+07  8.00000000e+10  1.41428571e+08  4.40000000e+11
  -2.10000000e+08  1.40000000e+11]
 [ 0.00000000e+00  0.00000000e+00 -2.10000000e+05 -2.10000000e+08
   2.10000000e+05 -2.10000000e+08]
 [ 0.00000000e+00  0.00000000e+00  2.10000000e+08  1.40000000e+11
  -2.10000000e+08  2.80000000e+11]] 

Displacements:
 [[ 0.00000000e+00]
 [ 2.27272727e-05]
 [ 0.00000000e+00]
 [-4.54545455e-05]
 [ 0.00000000e+00]
 [ 2.27272727e-05]] 

Forces:
 [[-1.55844156e+03]
 [ 2.64956616e-05]
 [-3.21428571e+03]
 [-1.50000000e+07]
 [ 4.77272727e+03]
 [-2.10665166e-06]]


# Problema 7.2

In [31]:
NodesCondition = []
Nodes = 5
Nodes *= 2
NumberOfElement = 4

E = 2.1e5 #MPA
K = []
I = 5e-5*1e12 #mm^4
w = -10 #N/mm
PosNodes = 1e3*np.array([0,3,6,8,10])
Elements = np.array([(0,1),(1,2),(2,3),(3,4)])
L = []
for i in range(NumberOfElement):
    L.append(PosNodes[Elements[i][1]] - PosNodes[Elements[i][0]])
L = np.array(L)

for i in range(0,NumberOfElement):
    aux = ElementStiffness(E,I,L[i])
    K.append(aux)


StiffnessMatrix = np.zeros((Nodes,Nodes))

U = np.zeros(Nodes).reshape(Nodes,1)
F = np.zeros(Nodes).reshape(Nodes,1)

Initialize(StiffnessMatrix,U,F)

#Node in UBoundary = Node*2+(y=0,phi=1)
UBoundaryCondition(U,0,2*0+0) #Nodo 0 en Y
UBoundaryCondition(U,0,2*0+1) #Nodo 0 en phi
UBoundaryCondition(U,0,2*1+0) #Nodo 1 en Y
UBoundaryCondition(U,0,2*2+0) #Nodo 2 en Y
UBoundaryCondition(U,0,2*4+0) #Nodo 4 en Y


FBoundaryCondition(F,-30e3,2*3+0) #M en Nodo 3
DFBoundaryCondition(F,w,0) #w distribuido en el elemento 0

U,F=Solve(StiffnessMatrix,U,F)
print("Stiffness Matrix:\n",StiffnessMatrix,'\n')
print("Displacements:\n",U,'\n')
print("Forces:\n",F)

Stiffness Matrix:
 [[ 4.66666667e+03  7.00000000e+06 -4.66666667e+03  7.00000000e+06
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.00000000e+06  1.40000000e+10 -7.00000000e+06  7.00000000e+09
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-4.66666667e+03 -7.00000000e+06  9.33333333e+03  0.00000000e+00
  -4.66666667e+03  7.00000000e+06  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.00000000e+06  7.00000000e+09  0.00000000e+00  2.80000000e+10
  -7.00000000e+06  7.00000000e+09  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -4.66666667e+03 -7.00000000e+06
   2.04166667e+04  8.75000000e+06 -1.57500000e+04  1.57500000e+07
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  7.00000000e+06  7.00000000e+09
   8.75000000e+06  3.50000000e+10 -1.57500000e+07  1.05000000e+10
   0.00000000e